In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from tqdm import tqdm  # Visualize loop progress
# from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed
import openpyxl
import os
import requests
import yfinance as yf

import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

In [2]:
# tradingview lightweight-charts
from lightweight_charts import Chart
import asyncio
import nest_asyncio
# Apply nest_asyncio AT START to allow nested event loops (required for Jupyter)
nest_asyncio.apply()
#
from dotenv import load_dotenv

In [3]:
# load env vars
load_dotenv()

True

In [4]:
# #####---------------------#####s
# screen_date = '2025-03-07'
# screen_date = '2025-03-13'
screen_date = '2025-03-19'
# #####---------------------#####
# read pre market data from TradingView data via csv
trading_view_df = pd.read_csv(f"/Users/sudz4/Desktop/MATT-MATTHEW/matt-matthew/tv_screen_gap-up_{screen_date}.csv")

In [5]:
# print(trading_view_df.columns)

In [6]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()
# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume',
    'Exponential Moving Average (3) 1 day', 'Exponential Moving Average (7) 1 day'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    # stock filtering conditions >=< based on criteria
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes',
    'Exponential Moving Average (3) 1 day',
    'Exponential Moving Average (7) 1 day'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stocks found.")
display(smash_df.head(20))
# display(smash_df)
# # ALL SMASH
# display(smash_df)

21 stocks found.


,Symbol,Description,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Industry,Index,Sector,Exchange,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time,Analyst Rating,Technical Rating 5 minutes,Exponential Moving Average (3) 1 day,Exponential Moving Average (7) 1 day
0,LSE,"Leishen Energy Holding Co., Ltd.",Micro caps,47.232947,64.478764,2.220060e+08,13.040,12.78,Oilfield services/Equipment,"NASDAQ Composite, NASDAQ Industrials",Industrial services,NASDAQ,NaN,NaN,1.524997e+06,1493342.5,5.072043e+05,4.148591e+05,218.887964,0.363482,0.156865,786.373905,NaN,Neutral,9.849026,7.806223
1,STNE,StoneCo Ltd.,Mid caps,10.152284,9.746193,3.378784e+09,11.385,10.81,Regional banks,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Industrials, Mini-Russell 2000",Finance,NASDAQ,2025-03-18,2025-06-03,2.465952e+08,5791667.2,7.133241e+06,6.364088e+06,3.356545,0.691739,0.414588,6.399670,Buy,Buy,10.612195,10.087026
2,OLLI,"Ollie's Bargain Outlet Holdings, Inc.",Mid caps,7.944680,0.605694,6.734329e+09,109.900,99.66,Department stores,"NASDAQ Composite, S&P MidCap 400, Nasdaq US Small Cap Growth, Russell 3000, Russell 1000, NASDAQ Industrials",Retail trade,NASDAQ,2025-03-19,2025-06-04,6.111721e+07,1351988.1,1.071373e+06,1.126807e+06,1.940040,0.698295,0.439740,4.360272,Buy,Buy,105.379852,103.617516
3,HCM,HUTCHMED (China) Limited,Mid caps,5.953177,5.484950,2.613070e+09,16.300,15.77,Pharmaceuticals: major,"NASDAQ Composite, NASDAQ Biotechnology",Health technology,NASDAQ,2025-03-19,2025-08-04,1.703187e+08,80041.5,8.456610e+04,7.774287e+04,1.758735,0.399321,0.417068,2.926813,Strong buy,Buy,15.729726,15.634394
4,GLNG,Golar LNG Limited,Mid caps,3.424658,3.139269,3.893026e+09,37.230,36.14,Marine shipping,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Industrials, Mini-Russell 2000, PHLX Oil Service Sector",Transportation,NASDAQ,2025-02-27,2025-06-03,8.763687e+07,2409987.4,1.681386e+06,1.435589e+06,0.349363,2.273900,1.035926,0.566859,Strong buy,Strong buy,35.929894,34.991276
5,PAGS,PagSeguro Digital Ltd.,Mid caps,3.295311,1.647655,2.800024e+09,8.495,8.02,Miscellaneous commercial services,"Russell 2000, Russell 3000, Mini-Russell 2000",Commercial services,NYSE,2025-02-20,2025-05-29,2.083492e+08,3870844.2,4.416143e+06,5.356811e+06,0.965433,0.755200,0.405365,1.677285,Buy,Buy,8.212017,7.944645
6,MOS,Mosaic Company (The),Mid caps,3.198162,2.536765,8.861621e+09,27.960,27.89,Chemicals: agricultural,"S&P 500, S&P 500 Materials, Russell 3000, Russell 1000, S&P 500 ESG",Process industries,NYSE,2025-02-27,2025-04-30,3.147420e+08,5402214.7,6.464559e+06,5.177204e+06,0.903600,0.716657,0.226223,2.079859,Buy,Strong buy,27.271822,26.396724
7,SPR,"Spirit Aerosystems Holdings, Inc.",Mid caps,2.249186,0.532702,4.094347e+09,34.915,33.97,Aerospace & defense,"Russell 3000, Russell 1000",Electronic technology,NYSE,2025-02-28,2025-04-30,1.159606e+08,1203319.8,1.226670e+06,1.464599e+06,0.754757,1.114919,0.573558,1.590128,Neutral,Buy,34.268271,33.835440
8,SRPT,"Sarepta Therapeutics, Inc.",Mid caps,2.094098,0.707098,7.779547e+09,80.175,74.06,Pharmaceuticals: major,"NASDAQ Composite, S&P MidCap 400, Nasdaq US Mid Cap Growth, Russell 3000, Russell 1000, NASDAQ Biotechnology",Health technology,NASDAQ,2025-02-26,2025-04-30,9.114342e+07,2475015.1,1.481256e+06,1.140305e+06,1.428862,0.826787,0.303991,2.680547,Strong buy,Buy,83.432081,90.332563
9,QFIN,"Qifu Technology, Inc",Mid caps,1.734847,1.734847,7.695908e+09,47.700,47.50,Finance/Rental/Leasing,"NASDAQ Composite, NASDAQ Golden Dragon China, NASDAQ Real Estate and Other Financial Services",Finance,NASDAQ,2025-03-16,2025-05-21,1.438676e+08,1699110.4,1.546843e+06,1.368350e+06,0.608523,1.232248,0.687569,1.067170,Strong buy,Buy,46.622531,44.630916


In [7]:
# group and count by sector to new df called sector_count_df
sector_count_df = smash_df.groupby('Sector').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
# print the date and time of the screen
print(f"Screening date: {screen_date}")
print(f"Total stocks returned: {len(smash_df)}")
display(sector_count_df)

Screening date: 2025-03-19
Total stocks returned: 21


,Sector,Count
0,Commercial services,5
4,Finance,3
2,Consumer services,2
3,Electronic technology,2
5,Health technology,2
8,Retail trade,2
1,Consumer durables,1
6,Industrial services,1
7,Process industries,1
9,Technology services,1


In [8]:
smash_list = smash_df['Symbol'].tolist()
print(len(smash_list))
print(smash_list)

21
['LSE', 'STNE', 'OLLI', 'HCM', 'GLNG', 'PAGS', 'MOS', 'SPR', 'SRPT', 'QFIN', 'LCID', 'TBBK', 'IPGP', 'VRNS', 'PAYO', 'TAL', 'LNW', 'WRBY', 'ADT', 'LB', 'CZR']


In [9]:
# Create a filter for EMA3 > EMA7
smash_ema_df = smash_df['Exponential Moving Average (3) 1 day'] > smash_df['Exponential Moving Average (7) 1 day']

# Apply the filter to smash_df
filtered_smash_df = smash_df[smash_ema_df]

# Print the number of stocks that meet the criteria
print(f"Number of stocks where EMA3 > EMA7: {len(filtered_smash_df)}")
print("\nFiltered stocks:")
print(filtered_smash_df['Symbol'].tolist())

Number of stocks where EMA3 > EMA7: 18

Filtered stocks:
['LSE', 'STNE', 'OLLI', 'HCM', 'GLNG', 'PAGS', 'MOS', 'SPR', 'QFIN', 'LCID', 'TBBK', 'IPGP', 'VRNS', 'PAYO', 'TAL', 'ADT', 'LB', 'CZR']
